## Multiple Chain 快速入门

Runnables 可以轻松地用来串联多个 Chains，使用 RunnablePassthrough 将输出同时传给多条后继链。

```
     Input
      / \
     /   \
 Chain1 Chain2
     \   /
      \ /
      Combine
```

本指南展示如何使用 Runnable 实现多个 AI 关于相同话题的辩论：

```
    输入话题
       |
       |
    原始观点
      / |\
     /  |  \
 正面论述| 反面论述
     \  | /
      \ |/
     最终总结
```

In [1]:
# 导入相关模块，包括运算符、输出解析器、聊天模板、ChatOpenAI 和 运行器
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough

# 创建一个计划器，生成一个关于给定输入的论证
planner = (
    ChatPromptTemplate.from_template("生成关于以下内容的论点: {input}")
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

# 创建正面论证的处理链，列出关于基础回应的正面或有利的方面
arguments_for = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的正面或有利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建反面论证的处理链，列出关于基础回应的反面或不利的方面
arguments_against = (
    ChatPromptTemplate.from_template(
        "列出关于{base_response}的反面或不利的方面"
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 创建最终响应者，综合原始回应和正反论点生成最终的回应
final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "正面观点:\n{results_1}\n\n反面观点:\n{results_2}"),
            ("system", "给出批评后生成最终回应"),
        ]
    )
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

# 构建完整的处理链，从生成论点到列出正反论点，再到生成最终回应
chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

In [2]:
print(chain.invoke({"input": "股市进入牛市"}))

在讨论“股市进入牛市”时，正面和反面的观点都提供了重要的视角。以下是对两方面论点的批评及综合回应：

### 批评

1. **经济泡沫风险**：
   - 虽然牛市可能导致泡沫的形成，但并非所有牛市都必然伴随泡沫。理性分析和基本面研究可以帮助识别潜在的泡沫风险，投资者应当保持警惕并采取适当的风险管理策略。

2. **企业盈利的不确定性**：
   - 企业盈利的波动性确实存在，但历史数据显示，强劲的牛市通常伴随着长期的盈利增长。因此，关注基本面和行业趋势仍是投资者的重要策略。

3. **利率上升的潜在威胁**：
   - 利率上升可能对股市造成压力，但适度的利率调整也可能是经济健康的表现。投资者应关注通胀和利率变化的背景，以便更好地评估市场反应。

4. **投资者过度投机**：
   - 虽然牛市可能吸引投机行为，但这也为理性的长期投资者提供了买入机会。投资者应该坚持自己的投资策略，避免被市场情绪左右。

5. **市场技术指标的虚假信号**：
   - 技术分析确实有局限性，但结合基本面分析和市场动态，能够提高投资决策的准确性。投资者应综合多种分析工具，避免单一依赖。

6. **行业集中风险**：
   - 行业集中风险是一个现实问题，但投资组合的多样化可以有效降低这种风险。投资者应当采用适当的资产配置策略，以减少对单一行业的依赖。

7. **外部冲击和不确定性**：
   - 外部冲击是不可避免的，但市场的韧性和适应能力也不可忽视。长期投资者通常能够在波动中找到机会。

8. **政策反转的风险**：
   - 政策的变化确实会影响市场情绪，但投资者可以通过调整投资组合和策略来应对这种变化。灵活性和适应性是应对市场波动的关键。

### 最终回应

在评估股市进入牛市的论点时，正面观点强调了经济复苏、企业盈利增长、低利率环境等积极因素，而反面观点则提醒我们关注潜在的风险与不确定性。理解这两方面的观点，对于投资者做出明智决策至关重要。

投资者应采取综合的方法，既要关注市场的积极信号，也要警惕潜在的风险。通过对基本面和技术指标的深入分析，结合多样化的投资策略和风险管理措施，投资者可以在牛市中寻求机会，同时降低损失的风险。最终，理性的投资决策和灵活的应对策略将帮助投资者在波动的市场中保持稳健的投资表现。


#### 流式输出

In [3]:
## chain 最终输出经过了 StrOutputParser 处理，所以可以直接输出流式输出 s
for s in chain.stream({"input": "全球经济"}):
    print(s, end="", flush=True)

在讨论全球经济的正面和反面观点时，我们可以看到各个方面的复杂性和相互关联性。以下是对以上论点的综合回应：

### 综合回应

1. **全球化的复杂性**：全球化确实带来了经济增长和文化交流，但也伴随着文化同质化和环境问题。为了实现经济与文化的双赢，各国需要在全球化的过程中，重视保护和弘扬地方文化，同时制定更严格的环境保护政策，以确保可持续发展。

2. **科技进步与包容性**：科技的进步是推动生产力提升的重要因素，但数字鸿沟和就业不稳定是不可忽视的挑战。各国应加强教育和技能培训，确保所有人都能平等地享受到科技进步的红利，从而减少社会不平等和不安定性。

3. **可持续发展的双重目标**：在追求经济增长的同时，必须重视环境保护。各国应采取更有效的环境政策，平衡经济发展与生态保护，避免资源争夺引发的地缘政治紧张。

4. **货币政策的审慎管理**：货币政策在促进经济增长方面发挥了重要作用，但也带来了金融不稳定和不平等加剧的问题。各国央行需要在刺激经济和控制风险之间找到平衡，确保货币政策的可持续性。

5. **新兴市场的机遇与挑战**：新兴市场的崛起为全球经济注入活力，但也带来了对传统经济体的依赖风险。发达国家应与新兴市场建立更紧密的合作关系，以应对全球经济波动带来的挑战。

6. **贸易政策的协调**：保护主义政策虽然短期内可能保护某些行业，但长期来看会损害全球经济的整体发展。各国应积极推动自由贸易，促进国际合作，以实现共同繁荣。

7. **疫情后的复苏策略**：疫情后的经济复苏确实存在不平衡的问题，各国应采取灵活的财政政策，优先支持受影响严重的行业和地区，确保经济复苏的包容性和可持续性。

### 结论

全球经济是一个复杂的系统，各国在追求自身利益的同时，必须加强合作与协调，以应对共同面临的挑战。通过创新、教育和可持续发展，世界各国可以在经济增长的基础上，推动更公平和包容的全球经济体系。关键在于各国如何在面对机遇与挑战时，采取适当的政策和措施，以实现长远的可持续发展。

### Homework: 实现一个多链版本的代码生成，输入功能需求，输出2种（Python，Java）以上编程语言的代码实现。